In [11]:
import numpy as np
import tensorflow as tf
import codecs
from os import listdir
import nltk
from gensim.models import Word2Vec
from random import randint

In [12]:
dirrs = ['sentiment/train/', 'sentiment/test/']
sent = []
for dirr in dirrs:
    print dirr
    l = listdir(dirr+'pos')
    print 'pos'
    for r in l:
        t = codecs.open(dirr+'pos/'+r,'r',encoding='utf8').read()
        sent.append(nltk.word_tokenize(t))
    l = listdir(dirr+'neg')
    print 'neg'
    for r in l:
        t = codecs.open(dirr+'neg/'+r,'r',encoding='utf8').read()
        sent.append(nltk.word_tokenize(t))

sentiment/train/
pos
neg
sentiment/test/
pos
neg


In [13]:
m = Word2Vec.load('embedding.model')
data = []
for s in sent:
    a = [m[x] for x in s]
    data.append(a)
data = np.array(data)

In [28]:
def sample(d):
    r = randint(0, data.shape[0] - 1)
    t = d[r]
    x = t[:-1]
    y = t[1:]
    return np.array(x), np.array(y)

s = sample(data)
s[0].shape, s[1].shape

((74, 200), (74, 200))

In [52]:
# hyper-parameters
epochs = 3700
hidden_layer = 100
inp_out_size = 200

In [47]:
'''
debugging 
Wxh = tf.Variable([[1, 2], [3, 4], [5, 6], [7, 8]], dtype=tf.float32)
Whh = tf.Variable([[1, 2], [3, 4]], dtype=tf.float32)
Why = tf.Variable([[1, 2, 3], [4, 5, 6]], dtype=tf.float32)
'''
Wxh = tf.Variable(tf.random_uniform([inp_out_size, hidden_layer], maxval=0.9))
Whh = tf.Variable(tf.random_uniform([hidden_layer, hidden_layer], maxval=0.9))
Why = tf.Variable(tf.random_uniform([hidden_layer, inp_out_size], maxval=0.9))

In [48]:
def recurrence(prev, inp):
    i = tf.reshape(inp, shape=[1, -1])
    p = tf.reshape(prev, shape=[1, -1])
    h = tf.squeeze(tf.matmul(i, Wxh)) + tf.squeeze(tf.matmul(p, Whh))
    return h

In [49]:
a = tf.placeholder(shape=[None, inp_out_size], dtype=tf.float32)
b = tf.placeholder(shape=[None, inp_out_size], dtype=tf.float32)
# initial = tf.Variable([1, 2], dtype=tf.float32)
initial = tf.Variable(tf.random_normal([100]), dtype=tf.float32)
states = tf.nn.tanh(tf.scan(recurrence, a, initializer=initial))
outputs = tf.matmul(states, Why)
# loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(outputs, b))
loss = tf.sqrt(tf.reduce_mean(tf.square(tf.sub(outputs, b))))
optimize_op = tf.train.AdagradOptimizer(0.01).minimize(loss)

In [ ]:
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    for i in range(epochs):
        x, y = sample(data)
        l, s, _ = sess.run([loss, states, optimize_op], {a: x, b: y})
        if i % 100 == 0:
            print 'epoch {0}, loss = {1}'.format(i, l)
    
    x, y = sample(data)
    o = sess.run(outputs, {a: x, b: y})

epoch 0, loss = 44.9276161194
